# MicaSense Dual Camera Workflow

## 1. Restructure image data folders
The native folder structure obtained from the RedEdge-MX and RedEdge-MX-Blue cameras are inconvenient, where several acquisition subfolders (e.g. 000/, 001/, 002/ etc) exist within a SYNCXXXXSET directory. Each of these subfolders contain 200 acquisitions per band, hence 1000 tif files for the five bands of the RedEdge-MX/RedEdge-MX-Blue cameras. The following code reorganises the SYNCXXXXSET folders such that:
```
SYNCXXXXSET/
     |--> dat/
           |--> diag0.dat
           |--> gpslog0.dat
           |--> hostlog0.dat
           |--> paramlog0.dat
     |--> IMG_0000_B.tif
     |--> IMG_0001_B.tif
     ....
     |--> IMG_YYYY_B.tif

where B = 1 -> 5 for RedEdge-MX, and
      B = 6 -> 10 for RedEdge-MX-Blue
```

In [ ]:
from pathlib import Path
from micasense.restructure_dirs import restructure

# Specify path to MicaSense image data
base_path = Path("/home/user_ra/Documents/uav_data/20211126_WoodmanPoint")
data_path = base_path / "micasense"

# re-running restructure() on a given directory won't delete
# anything or do anything untoward
restructure(data_path / "red_cam")  # restructure red_cam folder
restructure(data_path / "blue_cam")  # restructure blue_cam folder

## 2. Create metadata yaml files
Create a convenient set of yaml files that store all the necessary band acquisition metadata for each image acquisition set. This is especially prudent for the DualCamera, which for one acquisition set has 10 tif files in two separate folders.

In [ ]:
from pathlib import Path
from micasense.acquisition_yamls import create_img_acqi_yamls

# Specify the path to acquisition image data
# BASE_PATH = Path("/home/user_ra/Documents/uav_data/woodman_point_20211029")
base_path = Path("/home/user_ra/Documents/uav_data/20211126_WoodmanPoint")
data_path =  base_path / "micasense"

# NOTE: 
# 1. create_img_acqi_yamls() assumes that the folder structure of the image data
#    has already been reorganised via micasense.restructure_dirs.restructure()
# 2. when opath = None, the yaml files are saved in the folder,
#    data_path / "metadata"
create_img_acqi_yamls(dpath=base_path, opath=None)

## 3. Appending Reach M2 PPK Lat/Lon/Height to metadata yaml's
If the RedEdge-MX trigger signal was integrated to the Reach M2, then PPK positioning can be added to the yaml metadata files. This step requires processing the Reach M2 data and obtaining the *_events.pos file from external software such as RTKLIB

In [ ]:
from pathlib import Path
from micasense.acquisition_yamls import append_reach2yaml

# Specify the yaml metadata and reach m2 base paths
base_path = Path("/home/user_ra/Documents/uav_data/20211126_WoodmanPoint")
ypath = base_path / "micasense" / "metadata"  # yaml's created above
rpath = base_path / "reach_m2" / "processed"  # Reach M2

# Specify the leap seconds to substract from the Reach M2 GPS time to obtain UTC time
leap_sec = 18  # see https://endruntechnologies.com/support/leap-seconds

ms_to_reach = {
    "SYNC0009SET": rpath / "reach_202111260212" / "reach_raw_202111260212_events.pos",
    "SYNC0014SET": rpath / "reach_202111260245" / "reach_raw_202111260245_events.pos",
    "SYNC0016SET": rpath / "reach_202111260440" / "reach_raw_202111260440_events.pos",
    "SYNC0018SET": rpath / "reach_202111260445" / "reach_raw_202111260445_events.pos",
    "SYNC0021SET": rpath / "reach_202111260517" / "reach_raw_202111260517_events.pos",
}

for key in ms_to_reach:
    append_reach2yaml(
        yaml_path=ypath / key, events_file=ms_to_reach[key], leap_sec=leap_sec
    )